In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

In [69]:
"""
File Reading
"""
a = []
for i in range(14,32):
    try:
        for j in range(100):
            with open("../TrainingData/TeslaTrainingData_2019-10-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                try:
                    a.append(str(file.read()))
                except Exception as e:
                    print(e)
                    print(file.name)
    except:
        print("10/"+str(i))
for i in range(1,22):
    try:
        for j in range(100):
            with open("../TrainingData/TeslaTrainingData_2019-11-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                try:
                    a.append(str(file.read()))
                except Exception as e:
                    print(file.name)
    except:
        print("11/"+str(i))
print(len(a))

10/19
10/20
10/26
10/27
11/2
11/3
11/9
11/10
11/16
11/17
2900


In [57]:
"""
Neural Network Model Creation
"""
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
x_train = np.array(a, dtype=np.str)
x_train = x_train.astype(str)
# print(x_train.shape)
y_train = np.zeros(2700)+0.9
print(y_train.shape)
import tensorflow as tf
embed_size = 300 
max_features = 50000 
maxlen = 100
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
model1 = tf.keras.Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen))
model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model1.add(GlobalMaxPool1D())
model1.add(Dropout(0.2))
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(8, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(2,activation='softmax'))
model1.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
model1.summary()
x_trainR = np.array(x_train)[:2200]
y_trainR = np.array(y_train)[:2200]
x_test = np.array(x_train)[2200:]
y_test = np.array(y_train)[2200:]
x_train = x_trainR
y_train = y_trainR
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

(2700,)
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 256)          439296    
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 256)               0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_25 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 32)       

In [58]:
"""
Formats data labels
"""
def toFinal(a):
    for i in range(len(a)):
        if a[i] > 0:
            a[i] = 1
        else:
            a[i] = 0

In [59]:
"""
Reads and formats data labels
"""
y_train = []
for i in range(14,32):
    if(i != 19 and i!=20 and i!=26 and i!=27):
        try:
            with open("../TrainingData/TeslaTrainingData_2019-10-"+str(i)+"/Tesla.csv") as file:
                j = file.read().split(',')[1]
                assert j is not None
                j = j.replace('\n','')
                for _ in range(100):
                    y_train.append(float(j))
        except:
            continue
for i in range(1,22):
    if(i != 2 and i!=3 and i!=9 and i!=10 and i!=16 and i!=17):
        try:
            with open("../TrainingData/TeslaTrainingData_2019-11-"+str(i)+"/Tesla.csv") as file:
                j = file.read().split(',')[1]
                assert j is not None
                j = j.replace('\n','')
                for _ in range(100):
                    y_train.append(float(j))
        except:
            continue
toFinal(y_train)
print(y_train)
y_train = np.array(y_train)
print(y_train)
y_test = y_train[2200:]
y_train = y_train[:2200]
print(y_train.shape)
print(x_train.shape)
x_train = x_train[:1600]
print(x_train.shape)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [60]:
print(np.count_nonzero(y_train))
print(y_train.size)

1200
1600


In [61]:
"""
Trains Model
"""
history = model1.fit(x_train, y_train, epochs=10, batch_size=20)

Train on 1600 samples
Epoch 1/10
1600/1600 [==============================] - 23s 14ms/sample - loss: 0.6278 - accuracy: 0.7244
Epoch 2/10
1600/1600 [==============================] - 19s 12ms/sample - loss: 0.5755 - accuracy: 0.7494
Epoch 3/10
1600/1600 [==============================] - 19s 12ms/sample - loss: 0.5819 - accuracy: 0.7475
Epoch 4/10
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5763 - accuracy: 0.7475
Epoch 5/10
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.5745 - accuracy: 0.7481
Epoch 6/10
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5711 - accuracy: 0.7494
Epoch 7/10
1600/1600 [==============================] - 20s 13ms/sample - loss: 0.5738 - accuracy: 0.7494
Epoch 8/10
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.5649 - accuracy: 0.7500
Epoch 9/10
1600/1600 [==============================] - 19s 12ms/sample - loss: 0.5649 - accuracy: 0.7500
Epoch 10/10
1600/1600 [=

In [63]:
def predict(x):
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(x))
    x =pad_sequences(tokenizer.texts_to_sequences(x), maxlen=maxlen)
    return model1.predict(x)

In [66]:
print(model1.weights)

[<tf.Variable 'embedding_7/embeddings:0' shape=(50000, 300) dtype=float32, numpy=
array([[-0.00324253,  0.00367036, -0.06012659, ...,  0.02395823,
        -0.01589693, -0.02220433],
       [ 0.03264853, -0.04042051,  0.02365656, ..., -0.00375896,
        -0.01809701,  0.04728856],
       [-0.0075111 ,  0.08314388, -0.04958259, ..., -0.05833594,
        -0.01143002,  0.03880596],
       ...,
       [-0.03846743, -0.01813477, -0.02916116, ..., -0.02910435,
         0.04318285,  0.02199477],
       [ 0.03141865,  0.02994383,  0.03526722, ...,  0.02763083,
        -0.02455854,  0.00853541],
       [ 0.03022898,  0.04083757,  0.03729815, ..., -0.01868541,
         0.0251061 ,  0.03213276]], dtype=float32)>, <tf.Variable 'bidirectional_3/forward_lstm_3/kernel:0' shape=(300, 512) dtype=float32, numpy=
array([[ 0.01804008,  0.01948716, -0.072503  , ...,  0.07543774,
        -0.03419763,  0.03013366],
       [-0.01505032, -0.00891814, -0.04675484, ...,  0.06681027,
         0.03331655,  0.03369